In [ ]:
# load data
import os
import pickle

folder_path = "../dimension_allocation/result"
prefix = "Llama"
suffix = ".pkl"

def find_pkl_files(folder_path, prefix, suffix):
    pkl_files = [f for f in os.listdir(folder_path) if f.startswith(prefix) and f.endswith(suffix)]
    return pkl_files
result = find_pkl_files(folder_path, prefix, suffix)

new_ppl = {}
for file_name in result:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        loaded_data = pickle.load(file)
    for rope_dim in loaded_data.keys():
        new_ppl[rope_dim] = loaded_data[rope_dim]

new_ppl = dict(sorted(new_ppl.items()))


In [ ]:
# kv_cache - ppl || kv_cache - rope_dim&lora_dim(kv_dim)
import numpy as np
import matplotlib.pyplot as plt
import math

start = 1
step = 4
end = 2
head_num = 32

data = {}
for rope_dim in range(start, end, step):
    data[rope_dim] = {}
    for lora_dim in new_ppl[rope_dim].keys():
        data[rope_dim][lora_dim + rope_dim*2*head_num] = new_ppl[rope_dim][lora_dim]

colors = plt.cm.viridis(np.linspace(0, 1, math.ceil((end-start)/step)))

plt.figure(figsize=(12, 6), dpi=500)
for idx, (rope_dim, lora_ppl) in enumerate(data.items()):
    x_values = list(lora_ppl.keys())
    y_values = list(lora_ppl.values())
    plt.plot(x_values, y_values, color=colors[idx], label=f"{rope_dim}")

plt.xlabel(f"kv_cache")
plt.ylabel(f"ppl")
plt.grid(True, linestyle='--', alpha=0.5)
plt.title("performance of different dimension allocation")

plt.legend(title='number of chunks', loc="upper left", bbox_to_anchor=(1, 1))
plt.show()